Current plan:
- Spam aggregation error parsing tests
- If they can't be parsed, they are usable as tests to see if my approach will make them parseable
- If they can, I'll have to make error visualizations for them
- Actually I'll have to make error visualizations for them anyway

In [ ]:
from sqlvis import vis
import sqlite3

conn = sqlite3.connect('../../data/shopping.db')

In [ ]:
# Retrieve the schema from the db connection
schema = vis.schema_from_conn(conn)

In [ ]:
schema

## This notebook is mainly me testing every possible way to misplace aggregation to see if anything can still be parsed.

## Note: aggregation on the left side of the WHERE is caught with a pop-up, but on the right side isn't.

In [ ]:
query = '''
SELECT c.cName, p.price
FROM customer AS c, purchase AS p
WHERE c.cID = SUM(p.cID)
GROUP BY c.cName
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

In [ ]:
query = '''
SELECT c.cName, p.price
FROM customer AS c, purchase AS p
WHERE c.cID = p.cID
AND p.price > AVG(p.price)
GROUP BY c.cName
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

The one below is actually valid! It is just... slightly odd. Why is the subquery titled 'Or'?

In [ ]:
query = '''
SELECT c.cName, SUM(p.price)
FROM customer AS c, purchase AS p
WHERE c.cID = p.cID
AND p.price > (SELECT AVG(p.price)
               FROM purchase AS p)
GROUP BY c.cName
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

One is right, one is not. Check that both are handled properly.

In [ ]:
query = '''
SELECT c.cName, SUM(p.price)
FROM customer AS c, purchase AS p
WHERE c.cID = AVG(p.cID)
GROUP BY c.cName
HAVING p.price > AVG(p.price)
'''

# Generate the visualization.
vis.visualize(query, schema)

cur = conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

An interesting conclusion here. Can it be run? Absolutely not. Can our parser still parse it? Yes, though in its original form it behaves strangely. Disappearing links, disappearing column markings, an erroneous 'Or' title for a subquery... Curious behavior indeed. 

In fact... That 'Or' subgraph may be an SQLVis bug possibly?